In [2]:
import os
import json
import pandas as pd
import numpy as np
import jinja2
pd.reset_option("display.max_rows")
pd.reset_option("display.max_columns")
pd.reset_option("display.width")
pd.set_option('display.max_colwidth', 100)  # default is 50

In [53]:
import os
import json
import pandas as pd

root_dirs = [
    "experiments/tvn_defs_nc",
]

data_list = []

for root_dir in root_dirs:
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.json'):
                json_path = os.path.join(subdir, file)
                try:
                    with open(json_path, 'r') as f:
                        data = json.load(f)
                    
                    # Extract experiment name: first part after "experiments/"
                    experiment_name = os.path.normpath(subdir).split(os.sep)
                    experiment_idx = experiment_name.index("experiments") + 1
                    experiment = experiment_name[experiment_idx]
                    
                    # Extract timestamp: immediate parent folder of JSON file
                    timestamp = os.path.basename(subdir)
                    
                    # Add to data
                    data["experiment"] = experiment
                    data["timestamp"] = timestamp
                    
                    data_list.append(data)
                except json.JSONDecodeError:
                    print(f"Failed to decode JSON: {json_path}")

# Create DataFrame
df = pd.DataFrame(data_list)

# Optional: filter out unwanted models
exclude_models = ['openai/o4-mini-high', 'openai/gpt-4o-mini']
df = df[~df['model'].isin(exclude_models)]

len(df)


490

In [54]:
pd.set_option('display.max_colwidth', 100)
df.iloc[0].to_frame()

,0
model,google/gemini-2.5-flash-preview-05-20
exercise,11_not_open
prompt,"You are asked to write a proof in the syntax of Waterproof, a theorem proving educational softwa..."
tutorial,# Waterproof Tutorial\n\n## 1. We conclude that\n\n### Example:\n\nLemma example_reflexivity :\n...
full_input,"Your proof seems to have some mistakes in it, as it does not compile correctly.\nHere is the err..."
exercise_content,"Lemma not_open : ¬ ([0,1) is _open_).\nProof."
output,"Proof.\nWe need to show that (¬ ([0,1) is _open_)).\nWe argue by contradiction.\nAssume that ([0..."
token_count,33300
input_tokens,32987
thinking_tokens,0


In [86]:
model_to_inspect = 'anthropic/claude-sonnet-4'

# Filter to the model of interest
df_model = df[df['model'] == model_to_inspect].copy()

# Define a composite run identifier
df_model['full_run_id'] = df_model['experiment'] + ' | ' + df_model['timestamp'] + ' | ' + df_model['run_id']

# Get unique runs
unique_full_run_ids = df_model['full_run_id'].unique()

print(f"Number of unique runs for model '{model_to_inspect}': {len(unique_full_run_ids)}")

# Optionally show the run IDs for inspection
for i, run in enumerate(unique_full_run_ids):
    print(f"{i}: {run}")


Number of unique runs for model 'anthropic/claude-sonnet-4': 4
0: tvn_defs_nc | 2025-06-08_23-56-01 | anthropic/claude-sonnet-4::11_not_open::1
1: tvn_defs_nc | 2025-06-08_23-49-00 | anthropic/claude-sonnet-4::3_11_2::1
2: tvn_defs_nc | 2025-06-09_00-04-29 | anthropic/claude-sonnet-4::4_9_1::1
3: tvn_defs_nc | 2025-06-08_22-31-32 | anthropic/claude-sonnet-4::3_11_4::1


In [64]:
# Select the Nth run (e.g. first one)
run_index = 3 # change this to pick different run
selected_full_run_id = unique_full_run_ids[run_index]

# Filter rows corresponding to this run
df_run = df_model[df_model['full_run_id'] == selected_full_run_id].sort_values(by='attempt').copy()

# Clean up error message
df_run['error_message'] = df_run['errors'].apply(
    lambda x: x.split('\n', 1)[1] if isinstance(x, str) and '\n' in x else ''
)

# Show trace table
df_run[['model', 'exercise', 'attempt', 'error_message', 'line_with_error']]


,model,exercise,attempt,error_message,line_with_error
382,anthropic/claude-sonnet-4,3_11_4,1,Error: [Focus] Wrong bullet *: Current bullet * is not finished.\n\n,"* Indeed, (3 < 4)."
357,anthropic/claude-sonnet-4,3_11_4,2,Error: [Focus] Wrong bullet -: Current bullet - is not finished.\n\n,"- Indeed, (3 < 4)."
334,anthropic/claude-sonnet-4,3_11_4,3,Error: [Focus] Wrong bullet *: Current bullet * is not finished.\n\n,"* Indeed, (3 < 4)."
431,anthropic/claude-sonnet-4,3_11_4,4,"Error: You cannot do this right now, follow the advice in the goal window.\n\n",* Obtain such a y.
467,anthropic/claude-sonnet-4,3_11_4,5,"Error: You cannot do this right now, follow the advice in the goal window.\n\n","* By (i) it holds that (∃ y > 0, ∀ x > y, f (x) > 7)."
415,anthropic/claude-sonnet-4,3_11_4,6,Error: Expected a single focused goal but 2 goals are focused.\n\n,Obtain such a y.
427,anthropic/claude-sonnet-4,3_11_4,7,"Error: You cannot do this right now, follow the advice in the goal window.\n\n",* Obtain such a y.
363,anthropic/claude-sonnet-4,3_11_4,8,Error: Expected a single focused goal but 2 goals are focused.\n\n,Obtain such a y.
337,anthropic/claude-sonnet-4,3_11_4,9,Error: Expected a single focused goal but 2 goals are focused.\n\n,We need to show that (f(c) + g(c) > 10).
307,anthropic/claude-sonnet-4,3_11_4,10,,None


In [65]:
from IPython.display import display, HTML
import difflib
import html

def show_proof_diff(df_run, attempt_number_1, attempt_number_2):
    # Extract proofs
    proof_1 = df_run[df_run['attempt'] == attempt_number_1]['output'].iloc[0]
    proof_2 = df_run[df_run['attempt'] == attempt_number_2]['output'].iloc[0]

    # Split into lines and compute diff
    proof_1_lines = proof_1.splitlines()
    proof_2_lines = proof_2.splitlines()

    diff = list(difflib.ndiff(proof_1_lines, proof_2_lines))

    # Process diff for left and right columns
    left_col = []
    right_col = []

    for line in diff:
        tag = line[:2]
        content = html.escape(line[2:])  # escape <, >, &, etc.

        if tag == '  ':
            left_col.append(f"<div>{content}</div>")
            right_col.append(f"<div>{content}</div>")
        elif tag == '- ':
            left_col.append(f"<div style='background-color:#441111;'>{content}</div>")
            right_col.append(f"<div style='background-color:#222222;'></div>")
        elif tag == '+ ':
            left_col.append(f"<div style='background-color:#222222;'></div>")
            right_col.append(f"<div style='background-color:#114411;'>{content}</div>")
        elif tag == '? ':
            # ignore helper lines like ^^^^^^ in diffs
            continue

    # Combine columns into HTML
    html_table = f"""
    <div style="display: flex; gap: 10px; font-family: monospace; background-color: #1e1e1e; color: white; padding: 10px;">
        <div style="flex: 1;">
            <div style="font-weight: bold; padding-bottom: 5px;">Attempt {attempt_number_1}</div>
            {''.join(left_col)}
        </div>
        <div style="flex: 1;">
            <div style="font-weight: bold; padding-bottom: 5px;">Attempt {attempt_number_2}</div>
            {''.join(right_col)}
        </div>
    </div>
    """
    display(HTML(html_table))


In [66]:
show_proof_diff(df_run, 9, 10)

In [67]:
total_cost = df['cost'].sum()
total_cost

np.float64(4.18207827)

### Success rate per model

In [87]:
# Group by model and run, compute per-run success (1 if any attempt succeeded)
per_run_success = df.groupby(['model', 'full_run_id'])['success'].max().reset_index()

# Aggregate per model: success_rate, total_runs, successful_runs
agg_success_rate = per_run_success.groupby('model')['success'].agg(['mean', 'count', 'sum']).reset_index()

# Rename columns for clarity
agg_success_rate.rename(columns={
    'mean': 'success_rate',
    'count': 'total_runs',
    'sum': 'successful_runs'
}, inplace=True)

# Convert success_rate to % for display
agg_success_rate['success_rate'] = agg_success_rate['success_rate'] * 100

# Optional: round to 1 decimal place
agg_success_rate['success_rate'] = agg_success_rate['success_rate'].round(1)

# Reorder columns: successful_runs, total_runs, success_rate, sort by success_rate descending
agg_success_rate = agg_success_rate[['model', 'successful_runs', 'total_runs', 'success_rate']].sort_values(by='success_rate', ascending=False)

# Display the table
agg_success_rate

,model,successful_runs,total_runs,success_rate
1,anthropic/claude-sonnet-4,3,4,75.0
5,google/gemini-2.5-flash-preview-05-20:thinking,2,4,50.0
9,x-ai/grok-3-mini-beta,1,3,33.3
8,openai/o4-mini,1,4,25.0
6,openai/gpt-4.1,1,4,25.0
4,google/gemini-2.5-flash-preview-05-20,1,4,25.0
0,anthropic/claude-3.7-sonnet:thinking,0,1,0.0
3,deepseek/deepseek-r1-0528,0,1,0.0
2,deepseek/deepseek-chat-v3-0324,0,1,0.0
7,openai/o3-mini,0,3,0.0


### Success rate per exercise

In [88]:
# Group by exercise and run, compute per-run success
per_run_success_ex = df.groupby(['exercise', 'full_run_id'])['success'].max().reset_index()

# Aggregate per exercise
agg_success_rate_ex = per_run_success_ex.groupby('exercise')['success'].agg(['mean', 'count', 'sum']).reset_index()

# Rename columns for clarity
agg_success_rate_ex.rename(columns={
    'mean': 'success_rate',
    'count': 'total_runs',
    'sum': 'successful_runs'
}, inplace=True)

# Convert success_rate to % and round
agg_success_rate_ex['success_rate'] = agg_success_rate_ex['success_rate'] * 100
agg_success_rate_ex['success_rate'] = agg_success_rate_ex['success_rate'].round(1)

# Reorder columns: successful_runs, total_runs, success_rate, sort by success_rate descending
agg_success_rate_ex = agg_success_rate_ex[['exercise', 'successful_runs', 'total_runs', 'success_rate']].sort_values(by='success_rate', ascending=False)

# Display the table
agg_success_rate_ex


,exercise,successful_runs,total_runs,success_rate
1,3_11_2,6,7,85.7
3,4_9_1,2,7,28.6
2,3_11_4,1,8,12.5
0,11_not_open,0,7,0.0


### Success rate per model and exercise

In [89]:
# Group by model, exercise, run — compute per-run success
per_run_success_grid = df.groupby(['model', 'exercise', 'full_run_id'])['success'].max().reset_index()

# Now pivot: model as rows, exercise as columns, value = mean per run (in %)
grid = per_run_success_grid.pivot_table(
    index='model',
    columns='exercise',
    values='success',
    aggfunc='mean'
) * 100

# Round nicely
grid = grid.round(1)

# Display the grid
grid


exercise,11_not_open,3_11_2,3_11_4,4_9_1
model,,,,
anthropic/claude-3.7-sonnet:thinking,NaN,NaN,0.0,NaN
anthropic/claude-sonnet-4,0.0,100.0,100.0,100.0
deepseek/deepseek-chat-v3-0324,NaN,NaN,0.0,NaN
deepseek/deepseek-r1-0528,NaN,NaN,0.0,NaN
google/gemini-2.5-flash-preview-05-20,0.0,100.0,0.0,0.0
google/gemini-2.5-flash-preview-05-20:thinking,0.0,100.0,0.0,100.0
openai/gpt-4.1,0.0,100.0,0.0,0.0
openai/o3-mini,0.0,0.0,NaN,0.0
openai/o4-mini,0.0,100.0,0.0,0.0


In [77]:
# Average output token counts per model
output_tokens_avg = df.groupby('model')['output_tokens'].mean().reset_index()
output_tokens_avg.rename(columns={'output_tokens': 'Avg Output Tokens'}, inplace=True)

# Average thinking token counts per model
thinking_tokens_avg = df.groupby('model')['thinking_tokens'].mean().reset_index()
thinking_tokens_avg.rename(columns={'thinking_tokens': 'Avg Thinking Tokens'}, inplace=True)

# Get the thinking_mode per model
thinking_mode_per_model = df[['model', 'thinking_mode']].drop_duplicates()

# Merge into the result
output_tokens_avg = output_tokens_avg.merge(thinking_mode_per_model, on='model')
output_tokens_avg = output_tokens_avg.merge(thinking_tokens_avg, on='model')

output_tokens_avg.sort_values("Avg Thinking Tokens")

,model,Avg Output Tokens,thinking_mode,Avg Thinking Tokens
1,anthropic/claude-sonnet-4,310.454545,False,0.000000
2,deepseek/deepseek-chat-v3-0324,451.880000,False,0.000000
6,openai/gpt-4.1,344.318182,False,0.000000
4,google/gemini-2.5-flash-preview-05-20,488.782895,False,0.000000
9,x-ai/grok-3-mini-beta,184.238095,True,2.142857
7,openai/o3-mini,241.766667,True,1660.466667
3,deepseek/deepseek-r1-0528,396.600000,True,3348.600000
8,openai/o4-mini,213.947368,True,3628.473684
5,google/gemini-2.5-flash-preview-05-20:thinking,485.206897,True,5721.827586
0,anthropic/claude-3.7-sonnet:thinking,275.333333,True,13130.333333


In [80]:
# Define cost per attempt: group by model → mean cost
cost_per_attempt = df.groupby('model')['cost'].mean().reset_index()

# Rename column for clarity
cost_per_attempt.rename(columns={'cost': 'Avg Cost per Attempt ($)'}, inplace=True)

# Optional: add company column
cost_per_attempt['company'] = cost_per_attempt['model'].str.split('/').str[0]

# Display the table
cost_per_attempt


,model,Avg Cost per Attempt ($),company
0,anthropic/claude-3.7-sonnet:thinking,0.223071,anthropic
1,anthropic/claude-sonnet-4,0.032707,anthropic
2,deepseek/deepseek-chat-v3-0324,0.007487,deepseek
3,deepseek/deepseek-r1-0528,0.012034,deepseek
4,google/gemini-2.5-flash-preview-05-20,0.004078,google
5,google/gemini-2.5-flash-preview-05-20:thinking,0.005217,google
6,openai/gpt-4.1,0.003463,openai
7,openai/o3-mini,0.016452,openai
8,openai/o4-mini,0.023990,openai
9,x-ai/grok-3-mini-beta,0.002131,x-ai


In [81]:
ratios = []

for company, group in cost_per_attempt.groupby('company'):
    group_sorted = group.sort_values('Avg Cost per Attempt ($)')
    
    if len(group_sorted) >= 2:
        cheaper = group_sorted.iloc[0]
        more_expensive = group_sorted.iloc[-1]

        ratio = more_expensive['Avg Cost per Attempt ($)'] / cheaper['Avg Cost per Attempt ($)']
        model_pair = f"{more_expensive['model']} vs {cheaper['model']}"

        ratios.append({
            'Company': company,
            'Model Pair': model_pair,
            'Cost Ratio': ratio
        })
    else:
        pass  # optionally report companies with only 1 model

# Convert to DataFrame
ratios_df = pd.DataFrame(ratios)
ratios_df


,Company,Model Pair,Cost Ratio
0,anthropic,anthropic/claude-3.7-sonnet:thinking vs anthropic/claude-sonnet-4,6.820207
1,deepseek,deepseek/deepseek-r1-0528 vs deepseek/deepseek-chat-v3-0324,1.607392
2,google,google/gemini-2.5-flash-preview-05-20:thinking vs google/gemini-2.5-flash-preview-05-20,1.279174
3,openai,openai/o4-mini vs openai/gpt-4.1,6.926801


In [83]:
# Success rate per model compared across a few levels of tutorial verbosity
df['tutorial_len'] = df['tutorial'].apply(lambda x: len(x) if isinstance(x, str) else 0)

tutorial_success = df.groupby(['model', 'tutorial_len'])['success'].mean().reset_index()
tutorial_success['success'] = (tutorial_success['success'] * 100).round(2)
tutorial_success.rename(columns={'success': 'Success Rate (%)', 'tutorial_len': 'Tutorial Length'}, inplace=True)

tutorial_success

,model,Tutorial Length,Success Rate (%)
0,anthropic/claude-3.7-sonnet:thinking,8665,0.00
1,anthropic/claude-sonnet-4,8665,13.64
2,deepseek/deepseek-chat-v3-0324,8665,0.00
3,deepseek/deepseek-r1-0528,8665,0.00
4,google/gemini-2.5-flash-preview-05-20,8665,0.66
5,google/gemini-2.5-flash-preview-05-20:thinking,8665,6.90
6,openai/gpt-4.1,8665,0.65
7,openai/o3-mini,8665,0.00
8,openai/o4-mini,8665,5.26
9,x-ai/grok-3-mini-beta,8665,4.76


In [84]:
# Success rate per model compared across a few levels of prompt verbosity
df['prompt_len'] = df['prompt'].apply(lambda x: len(x) if isinstance(x, str) else 0)

prompt_success = df.groupby(['model', 'prompt_len'])['success'].mean().reset_index()
prompt_success['success'] = (prompt_success['success'] * 100).round(2)
prompt_success.rename(columns={'success': 'Success Rate (%)', 'prompt_len': 'Prompt Length'}, inplace=True)

prompt_success

,model,Prompt Length,Success Rate (%)
0,anthropic/claude-3.7-sonnet:thinking,1422,0.00
1,anthropic/claude-sonnet-4,1422,13.64
2,deepseek/deepseek-chat-v3-0324,1422,0.00
3,deepseek/deepseek-r1-0528,1422,0.00
4,google/gemini-2.5-flash-preview-05-20,1422,0.66
5,google/gemini-2.5-flash-preview-05-20:thinking,1422,6.90
6,openai/gpt-4.1,1422,0.65
7,openai/o3-mini,1422,0.00
8,openai/o4-mini,1422,5.26
9,x-ai/grok-3-mini-beta,1422,4.76


In [90]:
# List of (full_run_id, model, exercise, sorted list of success flags per attempt)
run_attempts = df.sort_values(['full_run_id', 'attempt']).groupby('full_run_id').agg({
    'model': 'first',
    'exercise': 'first',
    'success': lambda x: list(x),  # get list of success flags
    'attempt': 'max'  # store max_attempts per run (for correct "not used k" logic)
}).reset_index()

# Determine global max k (max number of attempts used in any run)
max_k = df['max_attempts'].max()

# Compute pass@k columns
for k in range(1, max_k + 1):
    def pass_at_k(row):
        success_flags = row['success']
        used_attempts = len(success_flags)
        success_in_k = any(success_flags[:k])
        not_used_k = used_attempts < k
        return success_in_k or not_used_k
    
    run_attempts[f'pass@{k}'] = run_attempts.apply(pass_at_k, axis=1)

# Now compute per-model pass@k mean
passk_cols = [f'pass@{k}' for k in range(1, max_k + 1)]
passk_summary = run_attempts.groupby('model')[passk_cols].mean().reset_index()

# Convert to % and round
for col in passk_cols:
    passk_summary[col] = (passk_summary[col] * 100).round(2)

# Optional: rename columns to Pass@k (%) style
passk_summary.rename(columns={col: f'Pass@{col.split("@")[1]} (%)' for col in passk_cols}, inplace=True)

# Display
passk_summary


,model,Pass@1 (%),Pass@2 (%),Pass@3 (%),Pass@4 (%),Pass@5 (%),Pass@6 (%),Pass@7 (%),Pass@8 (%),Pass@9 (%),...,Pass@41 (%),Pass@42 (%),Pass@43 (%),Pass@44 (%),Pass@45 (%),Pass@46 (%),Pass@47 (%),Pass@48 (%),Pass@49 (%),Pass@50 (%)
0,anthropic/claude-3.7-sonnet:thinking,0.00,0.00,0.00,100.00,100.00,100.00,100.00,100.00,100.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,anthropic/claude-sonnet-4,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
2,deepseek/deepseek-chat-v3-0324,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,deepseek/deepseek-r1-0528,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
4,google/gemini-2.5-flash-preview-05-20,0.00,25.00,25.00,25.00,25.00,25.00,25.00,25.00,25.00,...,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0
5,google/gemini-2.5-flash-preview-05-20:thinking,0.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
6,openai/gpt-4.1,0.00,0.00,0.00,25.00,25.00,25.00,25.00,25.00,25.00,...,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0
7,openai/o3-mini,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
8,openai/o4-mini,25.00,25.00,25.00,50.00,50.00,75.00,75.00,75.00,75.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
9,x-ai/grok-3-mini-beta,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0


In [24]:
# TODO:
# thinking vs no thinking models (all)
# thinking vs no thinking on models that support both (fair)
# definition expanding comparison